In [1]:

import sys 
sys.path.append(r'C:\Users\sergiojesus\Desktop\Diogo2017\EQS_LOCAL\Bitbucket_eqs')
import analytics_stat.distributions as di

import warnings
warnings.filterwarnings('ignore')

import datetime
import scipy.stats as s
#import scipy.optimize as opt
#import scipy.special as special
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline
# %matplotlib notebook

import os
import json
import pandas as pd
import qgrid
from IPython.display import HTML, Image, display, clear_output
# display(HTML("<style>.container { width:95% !important; }</style>"))

# py.sign_in('diogo.antunes.goncalves', 'kt7T5JadNn2AhWk3DdPR')

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import traitlets
from tkinter import Tk, filedialog

display(HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
This report hides the raw code used for its implementation by default.
To hide/show the raw code, click <a href="javascript:code_toggle()">here</a>.'''))

display(HTML('''
<button id="do_run_all">Click to start</button>
<script>
$("#do_run_all").click(
    function () {
        $("#run_cell").click();
    }
);
</script>'''))
        

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab
from spacy.language import Language
import spacy
import re
import unidecode
import edit_distance_nltk
nlp = Language(Vocab()) #Spacy variables
tokenizer = Tokenizer(nlp.vocab)
def tokenization_process(string):
    """tokenizes one string, removing not used punctuation in portuguese language in the process"""
    aux = tokenizer(string)
    output = list()
    for index, word in enumerate(aux):
        if re.search('([A-Za-zÀ-ÿ]+(-|)[A-Za-zÀ-ÿ]+|[A-Za-zÀ-ÿ]+)', str(word)):
                output.append(str(re.search('([A-Za-zÀ-ÿ]+(-|)[A-Za-zÀ-ÿ]+|[A-Za-zÀ-ÿ]+)', str(word)).group(0)).lower())
    return output

def toptokens(Tfidfmatrix, tokens, n=20):
    npmatrix = Tfidfmatrix.toarray()
    npmatrix[npmatrix == 0] = np.nan
    npmeans = np.nanmean(npmatrix, axis = 0)
    ordered_ids = np.argsort(npmeans)[::-1] 
    clean_ids = ordered_ids[(np.isnan(npmeans).sum()):]
    top_ids = clean_ids[:n]
    toptoken = [(tokens[i], npmeans[i]) for i in top_ids]
    df = pd.DataFrame(toptoken)
    df.columns = ['feature', 'TF-IDF']
    return df

def join_series(series_list):
    num_elements = series_list[0].shape[0]
    string_list = list()
    for i in range(num_elements):
        new_string = str()
        for column, value in enumerate(series_list):
            new_string = new_string + unidecode.unidecode(series_list[column][i]) + ' '
        string_list.append(new_string)
    new_series = pd.Series(data = string_list, index = range(num_elements))
    return new_series

In [3]:
class Display:
    
    def __init__(self):
        
        # Select file button
        self._bsf = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bsf.add_traits(files=traitlets.traitlets.List())
        self._bsf.description = "Select File"
        self._bsf.icon = "square-o"
        self._bsf.on_click(self.select_excel)
        
        
        # Load Excel button
        self._blj = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._blj.description = "Load File"
        self._blj.icon = 'download'
        self._blj.on_click(self.load_excel)
        self._blj.disabled = True
                
        # Spreadsheet Dropdown
        style = {'description_width': 'initial'}
        self._dds = widgets.Dropdown(options=['Select file'], value='Select file', disabled=True, style=style,
                                     description='Spreadsheet:')
        
        # Column Selection
        self._csm = widgets.SelectMultiple(options=['Select table'], value =['Select table'], disabled=True)
        
        # Column Selection button
        self._bcs = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bcs.description = "Choose Columns"
        self._bcs.on_click(self.choose_columns)
        self._bcs.disabled = True
        
        # define layout
        box_layout = widgets.Layout(display='center',
                            flex_flow='row',
                            align_items='center',
                            border='None',
                            justify_content='space-between',
                            width='900px')
        
        self._box = widgets.Box(children=[self._bsf, self._dds, self._blj,
                                          self._csm, self._bcs], layout=box_layout, main_size = 10)
        display(self._box)
            
    def select_excel(self,b): 
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        
        new_file = filedialog.askopenfilename(multiple=True,filetypes = [("XLS",'*.xls')])
        if new_file != "":
            self._bsf.files = new_file
            self._bsf.description = "File Selected"
            self._bsf.icon = "check-square-o"
            self._bsf.button_style = "success"
            self._blj.disabled = False
            print('Selected file: {}'.format(self._bsf.files))
            file_path = self._bsf.files
            self._excel = pd.ExcelFile(file_path[0], on_demand = True)
            self._sheets = self._excel.sheet_names
            self._dds.options = self._sheets
            self._dds.disabled = False

    def load_excel(self, b):
        self._dataframe = pd.read_excel(self._bsf.files[0], sheet_name = self._dds.value) 
        self._blj.button_style = "success"
        display(self._dataframe.head())
        self._csm.disabled = False
        self._csm.options = list(self._dataframe.columns.values)
        self._csm.disabled = False
        self._bcs.disabled = False
    
    def choose_columns(self, b):
        self._documents = list()
        for element in self._csm.value:
            self._dataframe[element].fillna("", inplace=True)
            self._documents.append( self._dataframe[element])
        self._documents = join_series(self._documents)
        print(self._documents)
        

In [4]:
d = Display()

Box(children=(Button(description='Select File', icon='square-o', layout=Layout(height='50px', width='500px'), …

Selected file: ["C:/Users/sergiojesus/Desktop/Recursos/Dados/Incidentes e OT's - Pontes 114 a 146.2 MX6 e MX7.xls"]


,OT,Descrição OT,Relevante,recordkey,worklogid,Column1,Descrição Log,Descrição Longa Log,Estado,Tipo Trabalho,...,Activo,Descrição Ativo,Data Registo,Ano,Mês,Sector,Inío Intervenção,Reposição do Activo,Fim Intervenção,Incidendia
0,ALS100011,Ponte 144 em avaria e com falha de autolevel,n,ALS100011,25902.0,25902.0,NaN,Foi verificada a UPS e no local está sem alarm...,FECHADA,MC,...,32124V010020,Ponte Telescópica FP-TEK 144,2010-07-26,2010,7,ALSENGMAN-CTC-TME,2010-07-26 07:35:00.000,2010-07-26 07:50:00.000,2010-07-26 07:50:00.000,1014.0
1,ALS100048,varia da manga 146,n,ALS100048,21676.0,21676.0,safty edge actuado. recolocada a manga ficando...,Depois de verificações a varios sistemas de se...,FECHADA,MC,...,32124V010022,Ponte Telescópica FP-TEK 146.1,2010-07-26,2010,7,ALSENGMAN-CTC-TME,2010-07-26 18:15:00.000,2010-07-26 18:30:00.000,2010-07-26 18:32:00.000,1042.0
2,ALS100053,Avaria manga 143,n,ALS100053,21768.0,NaN,puxada a manga atrás e retirado o alarme fican...,NaN,FECHADA,MC,...,32124V010019,Ponte Telescópica FP-TEK 143,2010-07-27,2010,7,ALSENGMAN-CTC-TME,2010-07-27 07:33:00.000,2010-07-27 07:45:00.000,2010-07-27 07:45:00.000,1059.0
3,ALS1000715,Reporte de ponte 146.2 fora de parque,n,ALS1000715,179442.0,179442.0,Reporte de ponte 146.2 fora de parque,Foram verificadas as duas pontes 146.1 e146.2 ...,REALIZ,MC,...,32124V010023,Ponte Telescópica FP-TEK 146.2,2012-11-21,2012,11,ALSCMT-PSE,2012-11-21 07:22:00.000,NaN,2012-11-21 07:32:00.000,42300.0
4,ALS100100,Avaria Ponte Telescópica FP-TEK 146.2,n,ALS100100,21774.0,21774.0,o operador tinha dificuldade em manobrar a manga,foi dada alguma informação de como proceder fi...,FECHADA,MC,...,32124V010023,Ponte Telescópica FP-TEK 146.2,2010-07-27,2010,7,ALSENGMAN-CTC-TME,2010-07-27 12:00:00.000,2010-07-27 12:20:00.000,2010-07-27 12:20:00.000,1073.0


0       Ponte 144 em avaria e com falha de autolevel  ...
1       varia da manga 146 safty edge actuado. recoloc...
2       Avaria manga 143 puxada a manga atras e retira...
3       Reporte de ponte 146.2 fora de parque Reporte ...
4       Avaria Ponte Telescopica FP-TEK 146.2 o operad...
5       Avaria Ponte Telescopica FP-TEK 146.2 o operad...
6       ALARME GERAL NA MANGA 145 retirado alarme fica...
7       ALARME GERAL NA PONTE 142 retirado alarme fica...
8       ALARME GERAL NA PONTE 142 retirado alarme fica...
9       Manga 146  com anomalia corrigida anomalia Sol...
10      Camara da manga virada ao contrario Ver Coment...
11      Camara da manga virada ao contrario Relatorio ...
12      Camara da manga virada ao contrario Relatorio ...
13      Manga 115 - Falha de Auto Level CORRIGIDA ANOM...
14      Ponte 146.2 ao finalizar acostamento em manual...
15                      PMR SPLITS MANGAS TELESCOPICAS   
16      Ponte 122 em avaria Ponte 122 em avaria Foi ne...
17      Ponte 

In [6]:
counter = CountVectorizer(tokenizer=tokenization_process)
count_matrix = counter.fit_transform(d._documents)
terms = np.array(counter.get_feature_names())
#tokens = toptokens(count_matrix, terms)
terms
index = np.array(range(len(terms)))

In [7]:
dicionario = pd.DataFrame.from_csv('Dicionario.csv')
dicionario.reset_index(inplace = True)
vec_dicionario = dicionario['Words'].values
for index, element in enumerate(vec_dicionario):
    vec_dicionario[index] = unidecode.unidecode(element.lower())

In [19]:
def word_checker(count_matrix, terms, repeats=100):
    #count_matrix em sparse
    #terms em np.array
    if len(terms) != count_matrix.shape[1]:
        raise ValueError("Terms and Count Matrix are not a match.")
    else:
        index_terms = np.array(range(len(terms)))
        token_frequency = count_matrix.sum(0)
        existent_words = terms[(token_frequency>repeats).A1]
        existent_words_index = index_terms[(token_frequency>repeats).A1]
        dubious_words = terms[(token_frequency<=repeats).A1]
        dubious_words_index = index_terms[(token_frequency<=repeats).A1]
        inexistent_words = np.array([])
        inexistent_words_index = np.array([])
        for number, term in enumerate(dubious_words):
            if term in vec_dicionario:
                existent_words = np.append(existent_words, term)
                existent_words_index = np.append(existent_words_index, dubious_words_index[number])
            else:
                inexistent_words = np.append(inexistent_words, term)
                inexistent_words_index = np.append(inexistent_words_index, dubious_words_index[number])
        return existent_words, inexistent_words, existent_words_index, inexistent_words_index

In [20]:
existent_words, inexistent_words, existent_words_index, inexistent_words_index =  word_checker(count_matrix, terms)

In [21]:
import time
begin = time.time()
new_list = [[edit_distance_nltk.edit_distance(i, j) for i in inexistent_words] for j in existent_words]
print(time.time()-begin)

1443.2269568443298


In [ ]:
len(terms)

In [18]:
len(existent_words_index)

5249

In [ ]:
np_list = np.array(new_list)

In [ ]:
np.save('comparison_list.npy', new_list)

In [22]:
best_match = np.argmin(new_list, axis = 0)
for inexistent, existent in enumerate(best_match):
    Correction_id = np.array()
    print(inexistent)
    print(inexistent_words[inexistent])
    print(existent_words[existent])

0
aaeronave
aeronave
1
aaf
aa
2
aalrme
alarme
3
aarme
alarme
4
aauto-level
auto-level
5
abadalhado
abandonado
6
abanado
abana
7
abandonamos
abandonadas
8
abandonei
abandonar
9
abanou
abana
10
abastecida
abatido
11
abatimentos
abatimento
12
abolsa
bolsa
13
about
aut
14
above
abre
15
abracadeiras
bracadeira
16
abrandou
abrindo
17
abrem-se
abrem
18
abreviados
retirados
19
abri-la
abril
20
abria
abra
21
abriam
abra
22
abruptamente
absolutamente
23
abrutamente
absolutamente
24
acabando
acabado
25
acabarem
acabar
26
acabava
acaba
27
acabe
acaba
28
acaitacao
aceitacao
29
acamponhada
acompanhada
30
acautele
acontece
31
acazevedo
acabado
32
accedez
aceder
33
accept
aceite
34
access
acesso
35
accionamento
funcionamento
36
accionando
accionado
37
acederam
aceder
38
acedi
aceder
39
acedido
cedido
40
aceesso
acesso
41
aceiracao
aceitacao
42
aceit
aceite
43
aceita-se
aceitacao
44
aceitaao
aceitacao
45
aceitabilidade
actividade
46
aceitacaode
aceitacao
47
aceitacap
aceitacao
48
aceitacdeslocou-se
des

387
amortecedores
fornecedor
388
amp
a
389
amperes
aerea
390
amribeiro
ribeiro
391
amstelveen
esteve
392
amt
cmt
393
analisada
analisado
394
analisarmos
analisado
395
analisei
analise
396
analist-analyft
analisado
397
analogica
logica
398
analogicos
analisou
399
anamolia
anomalia
400
anaso
acaso
401
ancostada
acostada
402
andrade
andre
403
andreia
andre
404
anexa-se
anexa
405
anexam
anexa
406
anexam-se
deram-se
407
anexas
anexa
408
angel
anel
409
angle
andre
410
animalia
anomalia
411
anobra
manobra
412
anolmalia
anomalia
413
anom
ano
414
anomalais
anomalia
415
anomalias-garantias
anomalias
416
anomlaia
anomalia
417
anomlia
anomalia
418
anomnalia
anomalia
419
anomola
anomala
420
anonyme
enorme
421
antepara
alterar
422
anteruiormente
anteriormente
423
anti-entalamento
nivelamento
424
anti-panico
antipanico
425
anti-queda
actuada
426
anti-retorno
retorno
427
antolin
antonio
428
antonieta
antonio
429
antunes
antes
430
anulada
aguada
431
anularam
anula
432
anutencao
manutencao
433
any
and
4

bride
774
bridg-in
bridge
775
bridge-in
bridge
776
bridgin
bridge
777
brig
ria
778
brige-in
bride
779
brin
ria
780
brindg
abrindo
781
broker
bumper
782
bsm
bom
783
btabarata
barata
784
bucins
buzina
785
bug
bag
786
bumber
bumper
787
bumpe
bumper
788
bumperes
bumper
789
bumpers
bumper
790
bumpoer
bumper
791
bumprer
bumper
792
bunper
bumper
793
buper
bumper
794
burner
bumper
795
busgate
resgate
796
but
aut
797
butoneira
botoneira
798
button
auto
799
buttons
bons
800
buzinas
buzina
801
by-pass
bypass
802
byp
by
803
bypasse
bypass
804
cabecote
cabeca
805
cabeleireiro
candeeiro
806
cabera
cabeca
807
cabinas
cabina
808
cabinete
gabinete
809
cablagem
colagem
810
cacostagem
acostagem
811
cacostamento
acostamento
812
caetano
cartao
813
caiam
caia
814
caiem
caem
815
cairam
cairem
816
caleiras
caldeiras
817
calendarizacao
calendario
818
calibracao
celebracao
819
calibrados
alterados
820
calisto
colisao
821
calmamente
claramente
822
camarate
camara
823
camopia
canopia
824
canalizadores
canalizada


1135
contactadoo
contacto
1136
contactados
contactos
1137
contactam
contacto
1138
contactar
contacto
1139
contactara
contacto
1140
contactarao
contacto
1141
contactato
contacto
1142
contacte
contacto
1143
contactor
contacto
1144
contactou
contacto
1145
contactou-nos
contactos
1146
contactou-se
contactos
1147
contagens
contagem
1148
contain
conta
1149
contained
contagem
1150
contains
contacto
1151
contatado
contratada
1152
contato
contacto
1153
contator
contador
1154
contemplada
controlada
1155
contemplado
cancelado
1156
contento
contendo
1157
contents
content
1158
contenus
contendo
1159
continou
continha
1160
continuando
continuado
1161
continuava
continuavam
1162
continuou
continua
1163
contract
contacto
1164
contracts
contacto
1165
contractual
contacto
1166
contrariando
contrarias
1167
contreras
contrarias
1168
contribuido
atribuido
1169
control
controle
1170
controladores
controlador
1171
controlodor
controlador
1172
convidando
considerando
1173
coord
cobre
1174
coordenacaovendas
co

desmontado
desmontar
1484
desmontagem
desmontar
1485
desncostagem
despistagem
1486
despejos
despejar
1487
desprimir
desinibir
1488
dessencostou-se
encostou-se
1489
destetou-se
deixou-se
1490
destinam-se
destinadas
1491
destinataire
destinatario
1492
destines
destinos
1493
destribuidas
atribuida
1494
destroy
dentro
1495
destroyed
dentro
1496
destruction
destruicao
1497
destruida
destruido
1498
desumidificador
purificador
1499
desviada
descida
1500
desviador
desviando
1501
detailed
detalhe
1502
detecao
deteccao
1503
detectada
detectar
1504
detectadas
detectar
1505
detectado
deteccao
1506
detectados
efectuados
1507
detectamos
deteccao
1508
detectando
deteccao
1509
detectasse
detectar
1510
detectato
deteccao
1511
detectavam
detectar
1512
detectei
detect
1513
detector
detectar
1514
detectores
detectar
1515
detectou
detect
1516
detectou-se
deixou-se
1517
deteriminou-se
deteriorar-se
1518
determinasse
determina
1519
determinou-se
determinadas
1520
detetada
deitada
1521
detetadas
deitada
1522


imagem
2424
imediacoes
imediato
2425
imediat
imediata
2426
imformou
informou
2427
immediately
imediata
2428
immediatement
imediatamente
2429
imobilizou-se
imobilizado
2430
imogti
imagem
2431
impeca
limpeza
2432
impecavelmente
especialmente
2433
impedem
impede
2434
impedia
impedir
2435
impediam
impedido
2436
impedida
impedido
2437
impeditivo
impedido
2438
impeditivos
impedido
2439
implementacao
implementado
2440
implementada
implementado
2441
implementadas
implementado
2442
implicacoes
explicacoes
2443
implicara
implicar
2444
implicava
implica
2445
implicou
implica
2446
importance
importante
2447
important
importante
2448
impossibilitando
impossibilidade
2449
impossibilitaram
impossibilitar
2450
impossibilitava
impossibilita
2451
impossibilite
impossibilita
2452
impossibilitou
impossibilita
2453
impossible
impossivel
2454
impreceptivel
imprescindivel
2455
imprima
implica
2456
imputados
imputado
2457
imputavel
imputacao
2458
imputs
impulso
2459
imqueiros
queiros
2460
inactivacao
inactivo

parqueada
3344
parqueado
parqueada
3345
parqueamento
arruamento
3346
parquearam
parqueada
3347
parqueasse
parqueada
3348
parqueda
parqueada
3349
parquei
parque
3350
parqueia
parque
3351
parqueou
parque
3352
parquing
parking
3353
part
para
3354
parti
parte
3355
parti-la
partida
3356
partindo-a
partida
3357
partiram
partida
3358
pasageiros
passageiros
3359
pasbarbosa
passados
3360
pass
pas
3361
passadeiras
passadeira
3362
passadoo
passado
3363
passageiros-terminal
passageiros
3364
passasse
passa
3365
passdo
passado
3366
passenger
assente
3367
passo-a
passa
3368
passou-se
mandou-se
3369
password
passar
3370
passwords
passos
3371
pasword
pascoal
3372
pataforma
plataforma
3373
patamas
patamar
3374
patricia
parecia
3375
pavimentada
alimentada
3376
pavimentada-zona
alimentacao
3377
paxs
pas
3378
pbb
pb
3379
pc
ac
3380
pd
d
3381
pde
de
3382
pdf
pdl
3383
pdoer
poder
3384
ped
pe
3385
peddo
pedido
3386
pedia
media
3387
pedida
pedido
3388
pedidod
pedido
3389
pedidoi
pedido
3390
pedimos
pedidos
339

sentor
sector
4307
sep
se
4308
sepam
sejam
4309
september
setembro
4310
sequei
sequer
4311
sergio
servico
4312
seriam
seria
4313
serr
ser
4314
serrao
serao
4315
server
serve
4316
service
servico4602
telm
tel
4603
telmo
mesmo
4604
tem-se
ter-se
4605
temdo
tempo
4606
tempe
tempo
4607
tempearatura
temperatura
4608
tempinho
tempo
4609
tempoexcedido
excedido
4610
temporiza
memoria
4611
temporizacao
memorizacao
4612
tendo-lhe
tendo-se
4613
tendo-me
tendo-se
4614
tensor
sensor
4615
tenta-se
tendo-se
4616
tentado
tentada
4617
tentasse
tendo-se
4618
tentavamos
estavamos
4619
tentei
teste
4620
tenter
tentar
4621
tentou-se
tendo-se
4622
tenttou-se
tendo-se
4623
tentyar
tentar
4624
teotonio
antonio
4625
tepo
tempo
4626
terda
perda
4627
teremos
termos
4628
teresea
teresa
4629
teriam
teria
4630
terminarado
terminado
4631
terminarem
terminar
4632
terminarmos
terminados
4633
terminasse
terminais
4634
terminassem
terminais
4635
terminava
terminada
4636
termindao
terminal
4637
termindos
terminados
4638


In [27]:
id_switch = np.array([[inexistent_words_index[inexistent].astype(int),existent_words_index[existent].astype(int)]
                       for inexistent, existent in enumerate(best_match)])

In [28]:
id_switch

array([[    2,   337],
       [    3,     1],
       [    4,   492],
       ...,
       [10169,   448],
       [10170,  1834],
       [10172,  7278]])

In [32]:
def tf_idf_error_remover(documents):
    timevariable = time.time()
    counter = CountVectorizer(tokenizer=tokenization_process)
    print(time.time()-timevariable)
    timevariable = time.time()
    count_matrix = counter.fit_transform(documents)
    print(time.time()-timevariable)
    timevariable = time.time()
    terms = np.array(counter.get_feature_names())
    print(time.time()-timevariable)
    timevariable = time.time()
    vec_dictionary = create_dictionary_nparray('Dicionario.csv')
    print(time.time()-timevariable)
    timevariable = time.time()
    existent_words, inexistent_words, existent_words_index, inexistent_words_index =  word_checker(count_matrix, terms)
    print(time.time()-timevariable)
    timevariable = time.time()
    distance_matrix = edit_distance_matrix(inexistent_words, existent_words) #bottleneck
    print(time.time()-timevariable)
    timevariable = time.time()
    id_switch = id_switcher_matrix(distance_matrix, existent_words_index, inexistent_words_index)


In [30]:
def edit_distance_matrix(first_term_vector, second_term_vector): #bottleneck
    return np.array([[edit_distance_nltk.edit_distance(i, j) for i in first_term_vector] for j in second_term_vector])

In [34]:
def id_switcher_matrix(distance_matrix, inexistent_words_index, existent_words_index):
    best_match = np.argmin(distance_matrix, axis = 0)
    id_switch = np.array([[inexistent_words_index[inexistent].astype(int),existent_words_index[existent].astype(int)]
                       for inexistent, existent in enumerate(best_match)])
    return id_switch

In [31]:
def create_dictionary_nparray(path):
    dictionary = pd.DataFrame.from_csv(path)
    dictionary.reset_index(inplace = True)
    vec_dictionary = dicionario['Words'].values
    for index, element in enumerate(vec_dictionary):
        vec_dictionary[index] = unidecode.unidecode(element.lower())
    return vec_dictionary

In [35]:
tf_idf_error_remover(d._documents)

0.0
3.183070182800293
0.009010076522827148
0.09833931922912598
10.393503665924072
1397.2030549049377


IndexError: index 5122 is out of bounds for axis 0 with size 5009